<center>CVPR, Finalterm Project, Spring 2022-23<br>
Title: An Extended ResUNet++ Architecture for Colonoscopic Image Segmentation<br>
</center>

In [1]:
import os
import numpy as np
import cv2
import random
import tensorflow as tf
from keras import layers
from keras.models import Model
from glob import glob
from scipy.ndimage import rotate
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.utils import Sequence
from keras import backend as K
from keras.metrics import Precision, Recall, MeanIoU
from keras.optimizers import Adam, Nadam, SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [2]:
def read_image(imagefile, grayscale=False):
    if grayscale == True:
        image = cv2.imread(imagefile)
        #image = np.expand_dims(image, -1)
    else:
        image = cv2.imread(imagefile)
    return image

def save_image(image, mask, path, binary=True):
    image = np.array(image)
    if binary == True:
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    cv2.imwrite(path[0], image)
    cv2.imwrite(path[1], mask)

def concat_images(images, rows, cols):
    _, h, w, _ = images.shape
    images = images.reshape((rows, cols, h, w, 3))
    images = images.transpose(0, 2, 1, 3, 4)
    images = images.reshape((rows * h, cols * w, 3))
    return images

def check_size(size):
    if type(size) == int:
        size = (size, size)
    if type(size) != tuple:
        raise TypeError('size is int or tuple')
    return size

def subtract(image):
    image = image / 255
    return image

def resize(image, size):
    size = check_size(size)
    image = cv2.resize(image, size)
    return image

def center_crop(image, mask, crop_size, size):
    h, w, _ = image.shape
    crop_size = check_size(crop_size)
    top = (h - crop_size[0]) // 2
    left = (w - crop_size[1]) // 2
    bottom = top + crop_size[0]
    right = left + crop_size[1]

    image = image[top:bottom, left:right, :]
    mask = mask[top:bottom, left:right, :]

    image = resize(image, size)
    mask = resize(mask, size)

    return image, mask

def random_crop(image, mask, crop_size, size):
    crop_size = check_size(crop_size)
    h, w, _ = image.shape
    top = np.random.randint(0, h - crop_size[0])
    left = np.random.randint(0, w - crop_size[1])
    bottom = top + crop_size[0]
    right = left + crop_size[1]

    image = image[top:bottom, left:right, :]
    mask = mask[top:bottom, left:right, :]

    image = resize(image, size)
    mask = resize(mask, size)

    return image, mask

def horizontal_flip(image, mask, size):
    image = image[:, ::-1, :]
    mask = mask[:, ::-1, :]

    image = resize(image, size)
    mask = resize(mask, size)

    return image, mask

def vertical_flip(image, mask, size):
    image = image[::-1, :, :]
    mask = mask[::-1, :, :]

    image = resize(image, size)
    mask = resize(mask, size)

    return image, mask

def scale_augmentation(image, mask, scale_range, crop_size, size):
    scale_size = np.random.randint(*scale_range)
    image = cv2.resize(image, (scale_size, scale_size))
    mask = cv2.resize(mask, (scale_size, scale_size))
    image, mask = random_crop(image, mask, crop_size, size)
    return image, mask

def random_rotation(image, mask, size, angle_range=(0, 90)):
    h1, w1, _ = image.shape
    h2, w2, _ = mask.shape

    angle = np.random.randint(*angle_range)
    image = rotate(image, angle)
    image = resize(image, (h1, w1))

    mask = rotate(mask, angle)
    mask = resize(mask, (h2, w2))

    image = resize(image, size)
    mask = resize(mask, size)

    return image, mask

def cutout(image_origin, mask_origin, size, mask_value='mean'):
    image = np.copy(image_origin)
    mask = np.copy(mask_origin)

    if mask_value == 'mean':
        mask_value = image.mean()
    elif mask_value == 'random':
        mask_value = np.random.randint(0, 256)

    h, w, _ = image.shape
    top = np.random.randint(0 - size // 2, h - size)
    left = np.random.randint(0 - size // 2, w - size)
    bottom = top + size
    right = left + size
    if top < 0:
        top = 0
    if left < 0:
        left = 0

    image[top:bottom, left:right, :].fill(mask_value)
    mask[top:bottom, left:right, :].fill(0)

    image = resize(image, size)
    mask = resize(mask, size)

    return image, mask

def brightness_augment(img, mask, size, factor=0.5):
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV) #convert to hsv
    hsv = np.array(hsv, dtype=np.float64)
    hsv[:, :, 2] = hsv[:, :, 2] * (factor + np.random.uniform()) #scale channel V uniformly
    hsv[:, :, 2][hsv[:, :, 2] > 255] = 255 #reset out of range values
    rgb = cv2.cvtColor(np.array(hsv, dtype=np.uint8), cv2.COLOR_HSV2RGB)

    image = resize(rgb, size)
    mask = resize(mask, size)

    return image, mask

def rgb_to_grayscale(img, mask, size):
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img = [img, img, img]
    img = np.transpose(img, (1, 2, 0))

    image = resize(img, size)
    mask = resize(mask, size)
    return image, mask

def create_dir(name):
    try:
        if not os.path.exists(name):
            os.mkdir(name)
    except:
        pass


In [4]:
# Augmentation
size = (256, 256)
crop_size = (300, 300)

path = "/kaggle/input/kvasirseg"
full_path = os.path.join(path, "Kvasir-SEG/")

new_path = "/kaggle/working/kvasirseg/New_Kvasir-SEG/"
create_dir(new_path)
new_full_path = os.path.join(new_path, "Kvasir-SEG/")

train_path = os.path.join(new_full_path, "train/")
valid_path = os.path.join(new_full_path, "valid/")
test_path = os.path.join(new_full_path, "test/")

if not os.path.exists(new_full_path):
    os.mkdir(new_full_path)
    for path in [train_path, valid_path, test_path]:
        os.mkdir(path)
        os.mkdir(os.path.join(path, "images/"))
        os.mkdir(os.path.join(path, "masks/"))

images = glob(os.path.join(full_path, "images", "*.*"))
masks = glob(os.path.join(full_path, "masks", "*.*"))

images.sort()
masks.sort()

len_ids = len(images)
train_size = int((80/100)*len_ids)
valid_size = int((10/100)*len_ids)
test_size = int((10/100)*len_ids)

train_images, test_images = train_test_split(images, test_size=test_size, random_state=42)
train_masks, test_masks = train_test_split(masks, test_size=test_size, random_state=42)

train_images, valid_images = train_test_split(train_images, test_size=test_size, random_state=42)
train_masks, valid_masks = train_test_split(train_masks, test_size=test_size, random_state=42)

print("Total Size: ", len_ids)
print("Training Size: ", len(train_images))
print("Validation Size: ", len(valid_images))
print("Testing Size: ", len(test_images))

# Testing images and masks resizing
for idx, p in tqdm(enumerate(test_images), total=len(test_images)): # p = path

    name = p.split("\\")[-1].split(".")[0]
    image_path = test_images[idx]
    mask_path = test_masks[idx]

    if os.path.exists(image_path) and os.path.exists(mask_path):
        image = read_image(image_path)
        mask = read_image(mask_path, grayscale=True)

        new_image_path = os.path.join(new_full_path, "test", "images/")
        new_mask_path = os.path.join(new_full_path, "test", "masks/")

        image = resize(image, size)
        mask = resize(mask, size)

        img_path = new_image_path + str(name) + ".jpg"
        mask_path = new_mask_path + str(name) + ".jpg"
        tmp_path = [img_path, mask_path]
        save_image(image, mask, tmp_path, binary=True)

# Validation images and masks resizing
for idx, p in tqdm(enumerate(valid_images), total=len(valid_images)):

    name = p.split("\\")[-1].split(".")[0]
    image_path = valid_images[idx]
    mask_path = valid_masks[idx]

    if os.path.exists(image_path) and os.path.exists(mask_path):
        image = read_image(image_path)
        mask = read_image(mask_path, grayscale=True)

        new_image_path = os.path.join(new_full_path, "valid", "images/")
        new_mask_path = os.path.join(new_full_path, "valid", "masks/")

        image = resize(image, size)
        mask = resize(mask, size)

        img_path = new_image_path + str(name) + ".jpg"
        mask_path = new_mask_path + str(name) + ".jpg"
        tmp_path = [img_path, mask_path]
        save_image(image, mask, tmp_path, binary=True)

# Training images and masks preprocessing
for idx, p in tqdm(enumerate(train_images), total=len(train_images)):

    name = p.split("\\")[-1].split(".")[0]
    image_path = train_images[idx]
    mask_path = train_masks[idx]

    if os.path.exists(image_path) and os.path.exists(mask_path):
        image = read_image(image_path)
        mask = read_image(mask_path, grayscale=True)

        # Augment the images and masks
        image1, mask1 = center_crop(image, mask, crop_size, size)
        image2, mask2 = random_crop(image, mask, crop_size, size)
        image3, mask3 = horizontal_flip(image, mask, size)
        image4, mask4 = vertical_flip(image, mask, size)
        image5, mask5 = scale_augmentation(image, mask, (512, 768), crop_size, size)
        image6, mask6 = random_rotation(image, mask, size)
        image7, mask7 = cutout(image, mask, 256)
        # Extra Cropping
        image8, mask8 = random_crop(image, mask, crop_size, size)
        image9, mask9 = random_crop(image, mask, crop_size, size)
        # Extra Scale Augmentation
        image10, mask10 = scale_augmentation(image, mask, (540, 820), crop_size, size)
        image11, mask11 = scale_augmentation(image, mask, (720, 1024), crop_size, size)
        # Extra Rotation
        image12, mask12 = random_rotation(image, mask, size)
        image13, mask13 = random_rotation(image, mask, size)
        # Brightness
        image14, mask14 = brightness_augment(image, mask, size, factor=0.3)
        image15, mask15 = brightness_augment(image, mask, size, factor=0.6)
        image16, mask16 = brightness_augment(image, mask, size, factor=0.9)
        # More Rotation
        image17, mask17 = random_rotation(image, mask, size)
        image18, mask18 = random_rotation(image, mask, size)
        # More Random Crop
        image19, mask19 = random_crop(image, mask, crop_size, size)
        image20, mask20 = random_crop(image, mask, crop_size, size)
        # More Cutout
        image21, mask21 = cutout(image, mask, 256)
        image22, mask22 = cutout(image, mask, 256)
        # Grayscale
        image23, mask23 = rgb_to_grayscale(image, mask, size)
        image24, mask24 = rgb_to_grayscale(image1, mask1, size)
        image25, mask25 = rgb_to_grayscale(image2, mask2, size)
        image26, mask26 = rgb_to_grayscale(image3, mask3, size)
        image27, mask27 = rgb_to_grayscale(image4, mask4, size)
        image28, mask28 = rgb_to_grayscale(image5, mask5, size)
        image29, mask29 = rgb_to_grayscale(image15, mask15, size)
        image30, mask30 = rgb_to_grayscale(image16, mask16, size)

        # Original image and mask
        image = resize(image, size)
        mask = resize(mask, size)

        # All images and masks
        all_images = [image, image1, image2, image3, image4, image5, image6, image7, 
                      image8, image9, image10, image11, image12, image13, image14, image15, image16, 
                      image17, image18, image19, image20, image21, image22, 
                      image23,image24, image25, image26, image27, image28, image29, image30]
        
        all_masks  = [mask, mask1, mask2, mask3, mask4, mask5, mask6, mask7, mask8, 
                      mask9, mask10, mask11, mask12, mask13, mask14, mask15, mask16, 
                      mask17, mask18, mask19, mask20, mask21, mask22, 
                      mask23, mask24, mask25, mask26, mask27, mask28, mask29, mask30]

        # Saving all the images and masks
        new_image_path = os.path.join(new_full_path, "train", "images/")
        new_mask_path = os.path.join(new_full_path, "train", "masks/")

        for j in range(len(all_images)):
            img_path = new_image_path + str(name) + "_" + str(j) + ".jpg"
            msk_path = new_mask_path + str(name) + "_" + str(j) + ".jpg"

            img = all_images[j]
            msk = all_masks[j]
            path = [img_path, msk_path]

            save_image(img, msk, path, binary=True)

In [3]:
def parse_image(img_path, image_size):
    image_rgb = cv2.imread(img_path, 1)
    h, w, _ = image_rgb.shape
    if (h == image_size) and (w == image_size):
        pass
    else:
        image_rgb = cv2.resize(image_rgb, (image_size, image_size))
    image_rgb = image_rgb.astype(np.float32)/255.0

    return image_rgb


def parse_mask(mask_path, image_size):
    mask = cv2.imread(mask_path, -1)
    h, w = mask.shape
    if (h == image_size) and (w == image_size):
        pass
    else:
        mask = cv2.resize(mask, (image_size, image_size))
    mask = np.expand_dims(mask, -1)
    mask = mask.astype(np.float32)/255.0

    return mask


class DataGen(Sequence):
    def __init__(self, image_size, images_path, masks_path, batch_size=8):
        self.image_size = image_size
        self.images_path = images_path
        self.masks_path = masks_path
        self.batch_size = batch_size
        self.on_epoch_end()

    def __getitem__(self, index):
        if(index+1)*self.batch_size > len(self.images_path):
            self.batch_size = len(self.images_path) - index*self.batch_size

        images_path = self.images_path[index*self.batch_size : (index+1)*self.batch_size]
        masks_path = self.masks_path[index*self.batch_size : (index+1)*self.batch_size]

        images_batch = []
        masks_batch = []

        for i in range(len(images_path)):                       # Read image and mask
            image = parse_image(images_path[i], self.image_size)
            mask = parse_mask(masks_path[i], self.image_size)

            images_batch.append(image)
            masks_batch.append(mask)

        return np.array(images_batch), np.array(masks_batch)

    def on_epoch_end(self):
        pass

    def __len__(self):
        return int(np.ceil(len(self.images_path)/float(self.batch_size)))

In [4]:
def squeeze_excite_block(inputs, ratio=8):
    init = inputs
    channel_axis = -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = layers.GlobalAveragePooling2D()(init)
    se = layers.Reshape(se_shape)(se)
    se = layers.Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = layers.Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)

    x = layers.Multiply()([init, se])
    return x



def stem_block(x, n_filter, strides):
    x_init = x

    x = layers.Conv2D(n_filter, (3, 3), padding="same", strides=strides)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Conv2D(n_filter, (3, 3), padding="same")(x)

    # Skip Connection
    s  = layers.Conv2D(n_filter, (1, 1), padding="same", strides=strides)(x_init)
    s = layers.BatchNormalization()(s)

    x = layers.Add()([x, s])
    x = squeeze_excite_block(x)
    return x



def resnet_block(x, n_filter, strides=1):
    x_init = x

    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Conv2D(n_filter, (3, 3), padding="same", strides=strides)(x)

    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Conv2D(n_filter, (3, 3), padding="same", strides=1)(x)

    # Skip Connection
    s  = layers.Conv2D(n_filter, (1, 1), padding="same", strides=strides)(x_init)
    s = layers.BatchNormalization()(s)

    x = layers.Add()([x, s])
    x = squeeze_excite_block(x)
    return x



def aspp_block(x, num_filters, rate_scale=1):
    x1 = layers.Conv2D(num_filters, (3, 3), dilation_rate=(6 * rate_scale, 6 * rate_scale), padding="same")(x)
    x1 = layers.BatchNormalization()(x1)

    x2 = layers.Conv2D(num_filters, (3, 3), dilation_rate=(12 * rate_scale, 12 * rate_scale), padding="same")(x)
    x2 = layers.BatchNormalization()(x2)

    x3 = layers.Conv2D(num_filters, (3, 3), dilation_rate=(18 * rate_scale, 18 * rate_scale), padding="same")(x)
    x3 = layers.BatchNormalization()(x3)

    x4 = layers.Conv2D(num_filters, (3, 3), padding="same")(x)
    x4 = layers.BatchNormalization()(x4)

    y = layers.Add()([x1, x2, x3, x4])
    y = layers.Conv2D(num_filters, (1, 1), padding="same")(y)
    return y



def attetion_block(s, x):
    """
        s: skip connection from the parallel encoder block
        x: output from ASPP/prev. decoder block
    """
    filters = x.shape[-1]

    s_conv = layers.BatchNormalization()(s)
    s_conv = layers.Activation("relu")(s_conv)
    s_conv = layers.Conv2D(filters, (3, 3), padding="same")(s_conv)

    s_pool = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(s_conv)

    x_conv = layers.BatchNormalization()(x)
    x_conv = layers.Activation("relu")(x_conv)
    x_conv = layers.Conv2D(filters, (3, 3), padding="same")(x_conv)

    xs_sum = layers.Add()([s_pool, x_conv])

    xs_conv = layers.BatchNormalization()(xs_sum)
    xs_conv = layers.Activation("relu")(xs_conv)
    xs_conv = layers.Conv2D(filters, (3, 3), padding="same")(xs_conv)

    xs_mul = layers.Multiply()([xs_conv, x])
    return xs_mul

In [5]:
def build_model(Input_Shape=(256, 256, 3)):
        
        n_filters = [16, 32, 64, 128, 256, 512]
        inputs = layers.Input(Input_Shape)

        c0 = inputs
        c1 = stem_block(c0, n_filters[0], strides=1)

        # Encoder Blocks
        c2 = resnet_block(c1, n_filters[1], strides=2)
        c3 = resnet_block(c2, n_filters[2], strides=2)
        c4 = resnet_block(c3, n_filters[3], strides=2)
        c5 = resnet_block(c4, n_filters[4], strides=2)

        # Atrous Spatial Pyramid Pooling Block
        b1 = aspp_block(c5, n_filters[5])

        # Decoder Blocks
        d0 = attetion_block(c4, b1)
        d0 = layers.UpSampling2D((2, 2))(d0)
        d0 = layers.Concatenate()([d0, c4])
        d0 = resnet_block(d0, n_filters[4])

        d1 = attetion_block(c3, d0)
        d1 = layers.UpSampling2D((2, 2))(d1)
        d1 = layers.Concatenate()([d1, c3])
        d1 = resnet_block(d1, n_filters[3])

        d2 = attetion_block(c2, d1)
        d2 = layers.UpSampling2D((2, 2))(d2)
        d2 = layers.Concatenate()([d2, c2])
        d2 = resnet_block(d2, n_filters[2])

        d3 = attetion_block(c1, d2)
        d3 = layers.UpSampling2D((2, 2))(d3)
        d3 = layers.Concatenate()([d3, c1])
        d3 = resnet_block(d3, n_filters[1])

        outputs = aspp_block(d3, n_filters[0])
        outputs = layers.Conv2D(1, (1, 1), padding="same")(outputs)
        outputs = layers.Activation("sigmoid")(outputs)

        model = Model(inputs, outputs)
        
        return model

In [6]:
smooth = 1.0

def dice_coef(y_true, y_pred):
    y_true_f = tf.keras.layers.Flatten()(y_true)
    y_pred_f = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2.0 * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)


def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [9]:

train_path = "/kaggle/input/aug-kvasir-seg/New_Kvasir-SEG/Kvasir-SEG/train"
valid_path = "/kaggle/input/aug-kvasir-seg/New_Kvasir-SEG/Kvasir-SEG/valid"

# Training
train_image_paths = glob(os.path.join(train_path, "images", "*.*"))
train_mask_paths = glob(os.path.join(train_path, "masks", "*.*"))
train_image_paths.sort()
train_mask_paths.sort()

train_image_paths = train_image_paths[:8000]
train_mask_paths = train_mask_paths[:8000]

# Validation
valid_image_paths = glob(os.path.join(valid_path, "images", "*.*"))
valid_mask_paths = glob(os.path.join(valid_path, "masks", "*.*"))
valid_image_paths.sort()
valid_mask_paths.sort()


image_size = 256
batch_size = 16
lr = 1e-4
epochs = 60

train_steps = len(train_image_paths)//batch_size
valid_steps = len(valid_image_paths)//batch_size


In [10]:
train_gen = DataGen(image_size, train_image_paths, train_mask_paths, batch_size=batch_size)
valid_gen = DataGen(image_size, valid_image_paths, valid_mask_paths, batch_size=batch_size)

In [12]:
model = build_model(Input_Shape=(image_size, image_size, 3))
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 16  448         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 256, 16  64         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [13]:
metrics = [Recall(), Precision(), dice_coef, MeanIoU(num_classes=2)]

model.compile(
    optimizer = Nadam(lr),
    loss = dice_loss,
    metrics = metrics
)

cp_path = "./cp_resunetpp_{epoch:03d}.h5"

checkpoint = ModelCheckpoint(cp_path, verbose=1, save_best_only=True, monitor='val_loss', save_freq='epoch')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-6, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
callbacks = [checkpoint, reduce_lr, early_stopping]

In [14]:
history = model.fit(
    train_gen,
    batch_size=batch_size,
    validation_data=valid_gen,
    steps_per_epoch=train_steps,
    validation_steps=valid_steps,
    epochs=epochs,
    callbacks=callbacks
)

Epoch 1/60
500/500 [==============================] - ETA: 0s - loss: 0.5705 - recall: 0.6987 - precision: 0.3677 - dice_coef: 0.4295 - mean_io_u: 0.4725
Epoch 1: val_loss improved from inf to 0.58996, saving model to ./cp_resunetpp_001.h5
500/500 [==============================] - 283s 505ms/step - loss: 0.5705 - recall: 0.6987 - precision: 0.3677 - dice_coef: 0.4295 - mean_io_u: 0.4725 - val_loss: 0.5900 - val_recall: 0.4214 - val_precision: 0.3939 - val_dice_coef: 0.4100 - val_mean_io_u: 0.4318 - lr: 1.0000e-04
Epoch 2/60
500/500 [==============================] - ETA: 0s - loss: 0.5306 - recall: 0.6508 - precision: 0.4245 - dice_coef: 0.4694 - mean_io_u: 0.5389
Epoch 2: val_loss did not improve from 0.58996
500/500 [==============================] - 240s 479ms/step - loss: 0.5306 - recall: 0.6508 - precision: 0.4245 - dice_coef: 0.4694 - mean_io_u: 0.5389 - val_loss: 0.5953 - val_recall: 0.6181 - val_precision: 0.3191 - val_dice_coef: 0.4047 - val_mean_io_u: 0.5663 - lr: 1.0000e-04

In [15]:
model.save('./ResUnetppm.h5')

In [7]:
from keras.models import load_model
from keras.utils import CustomObjectScope

In [8]:
def mask_to_3d(mask):
    mask = np.squeeze(mask)
    mask = [mask, mask, mask]
    mask = np.transpose(mask, (1, 2, 0))
    return mask

In [9]:
model_path = "D:/Github/CVPR/Finalterm/Project/ResUnetppm.h5"
test_path = "D:/Datasets/New_Kvasir-SEG/Kvasir-SEG/test/"

image_size = 256
batch_size = 1

test_image_paths = glob(os.path.join(test_path, "images", "*.*"))
test_mask_paths = glob(os.path.join(test_path, "masks", "*.*"))
test_image_paths.sort()
test_mask_paths.sort()

with CustomObjectScope({'dice_loss': dice_loss, 'dice_coef': dice_coef}):
    model = load_model(model_path)


In [10]:
# Testing
test_steps = len(test_image_paths)//batch_size
test_gen = DataGen(image_size, test_image_paths, test_mask_paths, batch_size=batch_size)
model.evaluate(test_gen, steps=test_steps, verbose=1)

100/100 [==============================] - 31s 295ms/step - loss: 0.3594 - recall: 0.4913 - precision: 0.9173 - dice_coef: 0.6406 - mean_io_u: 0.6462


[0.3594329059123993,
 0.49132952094078064,
 0.9173136353492737,
 0.6405670046806335,
 0.6462481021881104]

In [20]:
# Generating the result
for i, path in tqdm(enumerate(test_image_paths), total=len(test_image_paths)):
    image = parse_image(test_image_paths[i], image_size)
    mask = parse_mask(test_mask_paths[i], image_size)

    predict_mask = model.predict(np.expand_dims(image, axis=0))[0]
    predict_mask = (predict_mask > 0.5) * 255.0

    sep_line = np.ones((image_size, 10, 3)) * 255

    mask = mask_to_3d(mask)
    predict_mask = mask_to_3d(predict_mask)

    all_images = [image * 255, sep_line, mask * 255, sep_line, predict_mask]
    cv2.imwrite(f"./{i}.png", np.concatenate(all_images, axis=1))

  0%|          | 0/100 [00:00<?, ?it/s]

1/1 [==============================] - 1s 1s/step


  1%|          | 1/100 [00:01<02:31,  1.53s/it]

1/1 [==============================] - 0s 42ms/step


  3%|▎         | 3/100 [00:01<00:45,  2.14it/s]

...

 96%|█████████▌| 96/100 [00:11<00:00,  9.38it/s]

1/1 [==============================] - 0s 43ms/step


 98%|█████████▊| 98/100 [00:11<00:00,  9.77it/s]

1/1 [==============================] - 0s 39ms/step


100%|██████████| 100/100 [00:11<00:00,  8.36it/s]
